In [1]:
import boto3
from datetime import date
import time
import uuid
from credentials import *
import unittest

## Connect to DynamoDB

Initially, we connect to our local database we started as "dynamodb" container:

In [2]:
client = boto3.client('dynamodb', 
  endpoint_url='http://dynamo:8000', 
  region_name='eu-central-1',
  aws_access_key_id=aws_access_key_id,
  aws_secret_access_key=aws_secret_access_key
)
client.list_tables()

{'TableNames': ['twitter'],
 'ResponseMetadata': {'RequestId': 'cdfe43b0-d3d6-4ad9-8e50-118c48a03249',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Jan 2024 23:11:47 GMT',
   'x-amzn-requestid': 'cdfe43b0-d3d6-4ad9-8e50-118c48a03249',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2569046870',
   'content-length': '26',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

## Create Table

Let's setup the table with our attributes and indexes:

In [3]:
def create_app_table():
    return client.create_table(
        TableName="twitter",
        KeySchema=[
            {"AttributeName": "PK", "KeyType": "HASH"},
            {"AttributeName": "SK", "KeyType": "RANGE"},
        ],
        AttributeDefinitions=[
            {"AttributeName": "PK", "AttributeType": "S"},
            {"AttributeName": "SK", "AttributeType": "S"},
        ],
        ProvisionedThroughput={"ReadCapacityUnits": 10, "WriteCapacityUnits": 10},
    )

In [4]:
def delete_app_table():
    try:
        return client.delete_table(TableName="twitter")
    except:
        return False

In [5]:
delete_app_table()
create_app_table()

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'PK',
    'AttributeType': 'S'},
   {'AttributeName': 'SK', 'AttributeType': 'S'}],
  'TableName': 'twitter',
  'KeySchema': [{'AttributeName': 'PK', 'KeyType': 'HASH'},
   {'AttributeName': 'SK', 'KeyType': 'RANGE'}],
  'TableStatus': 'ACTIVE',
  'CreationDateTime': datetime.datetime(2024, 1, 11, 23, 11, 47, 405000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'LastIncreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'LastDecreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 10,
   'WriteCapacityUnits': 10},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/twitter',
  'DeletionProtectionEnabled': False},
 'ResponseMetadata': {'RequestId': '638b3670-f59c-4f38-8cd7-04a64db314e4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Jan 2024 23:11:47 GMT',
   'x-amz

## Helpers

To setup some other keys later on, we create a function to extract the id from a key:

In [6]:
def parse_id_from_key(key):
    return key.split("#")[-1]

parse_id_from_key("CHAN#7")

'7'

## Users

We create and save our users:

In [7]:
uuid_entry1 = str(uuid.uuid4())
user_martin = {
    'PK': {'S': f"USER#{uuid_entry1}"},
    'SK': {'S': f"USER#{uuid_entry1}"},
    'Name': {'S': 'Martin Marsal'},
}

uuid_entry2 = str(uuid.uuid4())
user_christian = {
    'PK': {'S': f"USER#{uuid_entry2}"},
    'SK': {'S': f"USER#{uuid_entry2}"},
    'Name': {'S': 'Christian Diegmann'},
}

uuid_entry3 = str(uuid.uuid4())
user_robin = {
    'PK': {'S': f"USER#{uuid_entry3}"},
    'SK': {'S': f"USER#{uuid_entry3}"},
    'Name': {'S': 'Robin Schüle'},
}

def save_user(user):
    return client.put_item(
        TableName="twitter",
        Item=user
    )

# AP6
def find_user(primaryKey, sortKey):
    item = client.get_item(
        TableName="twitter",
        Key={
          'PK': { 'S': primaryKey },
          'SK': { 'S': sortKey }
        }
    )
    return item['Item'] if 'Item' in item else False

In [8]:
save_user(user_martin)
find_user(user_martin['PK']['S'], user_martin['SK']['S'])

{'SK': {'S': 'USER#9a9941aa-4a2d-44ad-999c-e253c1b917e5'},
 'PK': {'S': 'USER#9a9941aa-4a2d-44ad-999c-e253c1b917e5'},
 'Name': {'S': 'Martin Marsal'}}

In [9]:
save_user(user_christian)
find_user(user_christian['PK']['S'], user_christian['SK']['S'])

{'SK': {'S': 'USER#96b97410-a549-430a-b6a4-048c1152f103'},
 'PK': {'S': 'USER#96b97410-a549-430a-b6a4-048c1152f103'},
 'Name': {'S': 'Christian Diegmann'}}

In [10]:
save_user(user_robin)
find_user(user_robin['PK']['S'], user_robin['SK']['S'])

{'SK': {'S': 'USER#87023191-51f0-4c49-bb25-fc13cce1a5a5'},
 'PK': {'S': 'USER#87023191-51f0-4c49-bb25-fc13cce1a5a5'},
 'Name': {'S': 'Robin Schüle'}}

In [11]:
def add_to_followers(primaryKey, follower):
    response = client.put_item(
        TableName="twitter",
        Item={
            'PK': {'S': primaryKey},
            'SK': {'S': f"FOLLOWER#{parse_id_from_key(follower['PK']['S'])}"},
            'Name': {'S': follower['Name']['S']},
        },
        )
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.get_item(
            TableName='twitter',
            Key={
                'PK': {'S': primaryKey},
                'SK': {'S': f"FOLLOWER#{parse_id_from_key(follower['PK']['S'])}"},
            }
        )

In [12]:
add_to_followers(user_martin['PK']['S'], user_christian)

{'Item': {'SK': {'S': 'FOLLOWER#96b97410-a549-430a-b6a4-048c1152f103'},
  'PK': {'S': 'USER#9a9941aa-4a2d-44ad-999c-e253c1b917e5'},
  'Name': {'S': 'Christian Diegmann'}},
 'ResponseMetadata': {'RequestId': '9a94596d-1fbb-4d25-9365-62f243c7aa41',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Jan 2024 23:11:47 GMT',
   'x-amzn-requestid': '9a94596d-1fbb-4d25-9365-62f243c7aa41',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '3243043190',
   'content-length': '158',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [13]:
add_to_followers(user_christian['PK']['S'], user_robin)

{'Item': {'SK': {'S': 'FOLLOWER#87023191-51f0-4c49-bb25-fc13cce1a5a5'},
  'PK': {'S': 'USER#96b97410-a549-430a-b6a4-048c1152f103'},
  'Name': {'S': 'Robin Schüle'}},
 'ResponseMetadata': {'RequestId': '0fab8c99-2d8d-46e4-af06-f47eb44fab33',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Jan 2024 23:11:47 GMT',
   'x-amzn-requestid': '0fab8c99-2d8d-46e4-af06-f47eb44fab33',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '3805266071',
   'content-length': '153',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [14]:
add_to_followers(user_robin['PK']['S'], user_martin)

{'Item': {'SK': {'S': 'FOLLOWER#9a9941aa-4a2d-44ad-999c-e253c1b917e5'},
  'PK': {'S': 'USER#87023191-51f0-4c49-bb25-fc13cce1a5a5'},
  'Name': {'S': 'Martin Marsal'}},
 'ResponseMetadata': {'RequestId': '3ae12a9b-ba7b-4b83-aa5b-5a44c0cab1ea',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Jan 2024 23:11:47 GMT',
   'x-amzn-requestid': '3ae12a9b-ba7b-4b83-aa5b-5a44c0cab1ea',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '18814152',
   'content-length': '153',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

## 1st access pattern: Post a tweet

In [15]:
uuid_entry4 = str(uuid.uuid4())
timestamp = int(time.time())
tweet = {'id': {'S': f"{uuid_entry4}"}, 'text': {'S': 'Moin, moin.'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}}
uuid_entry5 = str(uuid.uuid4())
timestamp = int(time.time())
tweet_2 = {'id': {'S': f"{uuid_entry5}"}, 'text': {'S': 'MongoDB ist super!'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}}
newMartin = find_user(user_martin['PK']['S'], user_martin['SK']['S'])

def post_tweet(primaryKey, tweet):
    client.put_item(
        TableName='twitter',
        Item={
            'PK': {'S': primaryKey},
            'SK': {'S': f"TWEET#{tweet['id']['S']}"},
            'id': {'S': tweet['id']['S']},
            'text': {'S': tweet['text']['S']},
            'likes': {'N': tweet['likes']['N']},
            'CreatedAt': { 'N': tweet['CreatedAt']['N']}
        },
    )
    item = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :follower)',
        ExpressionAttributeValues={
            ':pk': { 'S': primaryKey },
            ':follower': { 'S': 'FOLLOWER#' }
        },
    )
    followers = item['Items']
    for follower in followers:
        response2 = client.put_item(
            TableName='twitter',
            Item={
                'PK': {'S': f"USER#{parse_id_from_key(follower['SK']['S'])}"},
                'SK': {'S': f"TIMELINE#{tweet['id']['S']}"},
                'tweetId': {'S': tweet['id']['S']},
                'text': {'S': tweet['text']['S']},
                'likes': {'N': tweet['likes']['N']},
                'CreatedAt': { 'N': tweet['CreatedAt']['N']}
            },
        )
    if response2['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': f"USER#{parse_id_from_key(user_christian['PK']['S'])}" },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )

In [16]:
post_tweet(user_martin['PK']['S'], tweet)

{'Items': [{'SK': {'S': 'TIMELINE#5a03ab7c-daf3-4626-8fd9-00671049ca89'},
   'PK': {'S': 'USER#96b97410-a549-430a-b6a4-048c1152f103'},
   'text': {'S': 'Moin, moin.'},
   'tweetId': {'S': '5a03ab7c-daf3-4626-8fd9-00671049ca89'},
   'likes': {'N': '0'},
   'CreatedAt': {'N': '1705014707'}}],
 'Count': 1,
 'ScannedCount': 1,
 'ResponseMetadata': {'RequestId': '8d00aa9d-df93-43c4-be5d-5754f3cbc4aa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Jan 2024 23:11:47 GMT',
   'x-amzn-requestid': '8d00aa9d-df93-43c4-be5d-5754f3cbc4aa',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '3456900121',
   'content-length': '285',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [17]:
post_tweet(user_martin['PK']['S'], tweet_2)

{'Items': [{'SK': {'S': 'TIMELINE#36cd887d-e151-48be-a36c-2c239997d726'},
   'PK': {'S': 'USER#96b97410-a549-430a-b6a4-048c1152f103'},
   'text': {'S': 'MongoDB ist super!'},
   'tweetId': {'S': '36cd887d-e151-48be-a36c-2c239997d726'},
   'likes': {'N': '0'},
   'CreatedAt': {'N': '1705014707'}},
  {'SK': {'S': 'TIMELINE#5a03ab7c-daf3-4626-8fd9-00671049ca89'},
   'PK': {'S': 'USER#96b97410-a549-430a-b6a4-048c1152f103'},
   'text': {'S': 'Moin, moin.'},
   'tweetId': {'S': '5a03ab7c-daf3-4626-8fd9-00671049ca89'},
   'likes': {'N': '0'},
   'CreatedAt': {'N': '1705014707'}}],
 'Count': 2,
 'ScannedCount': 2,
 'ResponseMetadata': {'RequestId': '827fab4c-2edc-4e34-a402-343057f26f23',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Jan 2024 23:11:47 GMT',
   'x-amzn-requestid': '827fab4c-2edc-4e34-a402-343057f26f23',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2758427574',
   'content-length': '539',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [18]:
class TestPostTweet(unittest.TestCase):
    def test_post_tweet_enriched_timeline(self):
        christian_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_christian['PK']['S'] },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )
        timeline_length = len(christian_timeline['Items'])
        self.assertEqual(timeline_length, 2)

    def test_post_tweet_empty_timeline(self):
        robin_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_robin['PK']['S'] },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )
        timeline_length = len(robin_timeline['Items'])
        self.assertEqual(timeline_length, 0)

In [19]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestPostTweet)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_post_tweet_empty_timeline (__main__.TestPostTweet.test_post_tweet_empty_timeline) ... ok
test_post_tweet_enriched_timeline (__main__.TestPostTweet.test_post_tweet_enriched_timeline) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.017s

OK


## 2nd access pattern: Write a reply

In [20]:
uuid_entry6 = str(uuid.uuid4())
reply = {'id': {'S': f"{uuid_entry6}"}, 'text': {'S': 'Hallo zurück.'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}, 'tweetId': {'S': f"{parse_id_from_key(tweet['id']['S'])}"},}

def post_reply(author, user, tweet, reply):
    client.put_item(
        TableName='twitter',
        Item={
            'PK': {'S': f"USER#{parse_id_from_key(author['PK']['S'])}"},
            'SK': {'S': f"TWEET#{tweet['id']['S']}#{reply['id']['S']}"},
            'id': {'S': reply['id']['S']},
            'text': {'S': reply['text']['S']},
            'likes': {'N': reply['likes']['N']},
            'CreatedAt': { 'N': reply['CreatedAt']['N']}
        },
    )
    item = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :follower)',
        ExpressionAttributeValues={
            ':pk': { 'S': f"USER#{parse_id_from_key(author['PK']['S'])}" },
            ':follower': { 'S': 'FOLLOWER#' }
        },
    )
    followers = item['Items']
    for follower in followers:
        response2 = client.put_item(
            TableName='twitter',
            Item={
                'PK': {'S': f"USER#{parse_id_from_key(follower['SK']['S'])}"},
                'SK': {'S': f"TIMELINE#{tweet['id']['S']}#{reply['id']['S']}"},
                'id': {'S': reply['id']['S']},
                'text': {'S': reply['text']['S']},
                'likes': {'N': reply['likes']['N']},
                'CreatedAt': { 'N': reply['CreatedAt']['N']}
            },
        )
    if response2['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': f"USER#{parse_id_from_key(author['PK']['S'])}" },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )

In [21]:
post_reply(user_martin, user_christian, tweet, reply)

{'Items': [],
 'Count': 0,
 'ScannedCount': 0,
 'ResponseMetadata': {'RequestId': 'cfd93b2a-ae22-43de-a49a-555d957702e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Jan 2024 23:11:47 GMT',
   'x-amzn-requestid': 'cfd93b2a-ae22-43de-a49a-555d957702e8',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '583912891',
   'content-length': '39',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [22]:
class TestPostReply(unittest.TestCase):
    def test_post_reply_enriched_replies(self):
        test_tweet = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_christian['PK']['S'] },
                ':timeline': { 'S': f"TIMELINE#{tweet['id']['S']}" }
            },
        )
        tweet_length = len(test_tweet['Items'])
        self.assertEqual(tweet_length, 2)
        
    def test_post_reply_empty_replies(self):
        test_tweet = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_christian['PK']['S'] },
                ':timeline': { 'S': f"TIMELINE#{tweet_2['id']['S']}" }
            },
        )
        tweet_length = len(test_tweet['Items'])
        self.assertEqual(tweet_length, 1)

In [23]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestPostReply)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_post_reply_empty_replies (__main__.TestPostReply.test_post_reply_empty_replies) ... ok
test_post_reply_enriched_replies (__main__.TestPostReply.test_post_reply_enriched_replies) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.018s

OK


## 3rd access pattern: Edit a tweet

In [24]:
def edit_tweet(primaryKey, tweetId, new_text):

    response = client.update_item(
        TableName="twitter",
        Key={
            'PK': {'S': primaryKey},
            'SK': {'S': f"TWEET#{tweetId}"}
        },
        UpdateExpression='SET #text = :new_text',
        ExpressionAttributeNames={
            '#text': 'text'
        },
        ExpressionAttributeValues={
            ':new_text': {'S': new_text}
        }
    )


    followers_query = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :follower)',
        ExpressionAttributeValues={
            ':pk': {'S': primaryKey},
            ':follower': {'S': 'FOLLOWER#'}
        },
    )

    followers = followers_query.get('Items', [])
    for follower in followers:
        client.update_item(
            TableName="twitter",
            Key={
                'PK': {'S': f"USER#{parse_id_from_key(follower['SK']['S'])}"},
                'SK': {'S': f"TIMELINE#{tweetId}"}
            },
            UpdateExpression='SET #text = :new_text',
            ExpressionAttributeNames={
                '#text': 'text'
            },
            ExpressionAttributeValues={
                ':new_text': {'S': new_text}
            }
        )

    return response


In [25]:
new_tweet_text = "DynamoDB ist super"
edit_tweet(user_martin['PK']['S'], uuid_entry5, new_tweet_text)


{'ResponseMetadata': {'RequestId': 'da1f896e-65c7-4198-8738-3dd2b863de64',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Jan 2024 23:11:47 GMT',
   'x-amzn-requestid': 'da1f896e-65c7-4198-8738-3dd2b863de64',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2745614147',
   'content-length': '2',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [26]:
def get_user_tweets(primaryKey):
    response = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :tweet)',
        ExpressionAttributeValues={
            ':pk': {'S': primaryKey},
            ':tweet': {'S': 'TWEET#'}
        }
    )

    tweets = response.get('Items', [])
    return tweets

In [27]:
user_martin_tweets = get_user_tweets(user_martin['PK']['S'])
print(user_martin_tweets)

[{'SK': {'S': 'TWEET#36cd887d-e151-48be-a36c-2c239997d726'}, 'PK': {'S': 'USER#9a9941aa-4a2d-44ad-999c-e253c1b917e5'}, 'id': {'S': '36cd887d-e151-48be-a36c-2c239997d726'}, 'text': {'S': 'DynamoDB ist super'}, 'CreatedAt': {'N': '1705014707'}, 'likes': {'N': '0'}}, {'SK': {'S': 'TWEET#5a03ab7c-daf3-4626-8fd9-00671049ca89'}, 'PK': {'S': 'USER#9a9941aa-4a2d-44ad-999c-e253c1b917e5'}, 'id': {'S': '5a03ab7c-daf3-4626-8fd9-00671049ca89'}, 'text': {'S': 'Moin, moin.'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705014707'}}, {'SK': {'S': 'TWEET#5a03ab7c-daf3-4626-8fd9-00671049ca89#172be746-9d26-4818-a887-35080ac97f83'}, 'PK': {'S': 'USER#9a9941aa-4a2d-44ad-999c-e253c1b917e5'}, 'id': {'S': '172be746-9d26-4818-a887-35080ac97f83'}, 'text': {'S': 'Hallo zurück.'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705014707'}}]


In [28]:
class TestEditTweet(unittest.TestCase):
    def test_edit_tweet__timeline(self):
        tweet_id_to_find = uuid_entry5  
        updated_text = "DynamoDB ist super"

        christian_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND SK = :sk',
            ExpressionAttributeValues={
                ':pk': {'S': f"USER#{parse_id_from_key(user_christian['PK']['S'])}"},
                ':sk': {'S': f"TIMELINE#{tweet_id_to_find}"}
            },
        )
        matching_items = christian_timeline.get('Items', [])

        self.assertTrue(
            any('text' in item and item['text']['S'] == updated_text for item in matching_items),
            f"Tweet with ID {tweet_id_to_find} not found in timeline or text not updated."
        )

    def test_edit_tweet_martin(self):
        tweet_id_to_find = uuid_entry5  
        updated_text = "DynamoDB ist super"  

        
        martin_tweets = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND SK = :sk',
            ExpressionAttributeValues={
                ':pk': {'S': f"USER#{parse_id_from_key(user_martin['PK']['S'])}"},
                ':sk': {'S': f"TWEET#{tweet_id_to_find}"}
            },
        )
        matching_items_martin = martin_tweets.get('Items', [])

        # Check if the updated text is found in user_martin's tweets
        self.assertTrue(
            any('text' in item and item['text']['S'] == updated_text for item in matching_items_martin),
            f"Tweet with ID {tweet_id_to_find} not found in Martin's tweets or text not updated."
        )


    def test_edit_tweet_empty_timeline(self):
        robin_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_robin['PK']['S'] },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )
        timeline_length = len(robin_timeline['Items'])
        self.assertEqual(timeline_length, 0)

In [29]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestEditTweet)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_edit_tweet__timeline (__main__.TestEditTweet.test_edit_tweet__timeline) ... ok
test_edit_tweet_empty_timeline (__main__.TestEditTweet.test_edit_tweet_empty_timeline) ... ok
test_edit_tweet_martin (__main__.TestEditTweet.test_edit_tweet_martin) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.027s

OK


# 4th access pattern: Read a timeline

In [30]:
def read_timeline(primaryKey):
    timeline_query = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
        ExpressionAttributeValues={
            ':pk': {'S': primaryKey},
            ':timeline': {'S': 'TIMELINE#'}
        },
    )
    timeline_items = timeline_query.get('Items', [])
    
    formatted_timeline = [
        {
            'id': item.get('tweetId', item.get('id', 'N/A')),
            'text': item.get('text', 'N/A'),
            'likes': item.get('likes', 'N/A'),
            'CreatedAt': item.get('CreatedAt', 'N/A')
        }
        for item in timeline_items
    ]


    return formatted_timeline

In [31]:
user_christian_timeline = read_timeline(user_christian['PK']['S'])
print(user_christian_timeline)

[{'id': {'S': '36cd887d-e151-48be-a36c-2c239997d726'}, 'text': {'S': 'DynamoDB ist super'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705014707'}}, {'id': {'S': '5a03ab7c-daf3-4626-8fd9-00671049ca89'}, 'text': {'S': 'Moin, moin.'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705014707'}}, {'id': {'S': '172be746-9d26-4818-a887-35080ac97f83'}, 'text': {'S': 'Hallo zurück.'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705014707'}}]


In [32]:
class TestReadTimeline(unittest.TestCase):
    def test_read_timeline(self):
        # Assuming user_christian has a populated timeline
        expected_texts = ["Moin, moin.", "Hallo zurück.", "DynamoDB ist super"]

        # Fetch the timeline of user_christian
        christian_timeline = read_timeline(user_christian['PK']['S'])

        # Extract text items from the timeline
        actual_texts = [item['text']['S'] for item in christian_timeline]

        # Check if the actual text items match the expected texts (order doesn't matter)
        self.assertCountEqual(actual_texts, expected_texts, "Timeline text items do not match the expected texts.")

In [33]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestReadTimeline)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_read_timeline (__main__.TestReadTimeline.test_read_timeline) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.010s

OK


## Channel

We create our channel and functions to save, find and count users and messages in the channel:

In [34]:
channel_town_hall = {
    'PK': { 'S': "CHAN#7" },
    'SK': { 'S': "CHAN#7" },
    'Name': { 'S': 'Town Hall' },
    'Desc': { 'S': 'General News' },
    'UserCount': { 'N': "0" },
    'MessageCount': { 'N': "0" }
}

def save_channel(channel):
    return client.put_item(
        TableName="chat",
        Item=channel
    )

# AP2
def find_channel(key):
    item = client.get_item(
        TableName="chat",
        Key={
          'PK': { 'S': key },
          'SK': { 'S': key }
        }
    )
    return item['Item'] if 'Item' in item else False

# AP4
def message_count_for_channel(channel):
    return int(channel['MessageCount']['N'])

# AP7
def user_count_for_channel(channel):
    return int(channel['UserCount']['N'])

save_channel(channel_town_hall)
channel_town_hall = find_channel(channel_town_hall['PK']['S'])
channel_town_hall

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the PutItem operation: Cannot do operations on a non-existent table

In [ ]:
message_count_for_channel(channel_town_hall)

In [ ]:
user_count_for_channel(channel_town_hall)

## User Join

When a user joins a channel, we create a new item and increment the user counter. Further, we setup queries to retreive the users for a channel and channels of a user:

In [ ]:
def join_channel(user, channel):
    # create userj record
    client.put_item(
        TableName="chat",
        Item={
            'PK': channel['PK'],
            'SK': { 'S': "USERJ#" + parse_id_from_key(user['PK']['S']) },
            'Name': user['Name'],
            'JoinedAt': { 'S': str(date.today()) },
            'GSI1PK': user['PK'],
            'GSI1SK': channel['PK'],
            'ChanName': channel['Name']
        }
        )
    
    # increment users
    client.update_item(
        TableName="chat",
        Key = {
            'PK': channel['PK'],
            'SK': channel['PK'],
        },
        ExpressionAttributeValues = {
            ':one': { 'N': '1' }
        },
        UpdateExpression = 'ADD UserCount :one', 
        ReturnValues = 'UPDATED_NEW'
      )

# AP5
def users_in_channel(key):
    item = client.query(
        TableName="chat",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :userj)',
        ExpressionAttributeValues={
            ':pk': key,
            ':userj': { 'S': 'USERJ#' }
        }
    )
    return item['Items'] if 'Items' in item else []

# AP1
def channels_for_user(key):
    item = client.query(
        TableName="chat",
        IndexName="GSI1",
        KeyConditionExpression='GSI1PK = :pk AND begins_with(GSI1SK, :chan)',
        ExpressionAttributeValues={
            ':pk': key,
            ':chan': { 'S': 'CHAN#' }
        }
    )
    return item['Items'] if 'Items' in item else []

join_channel(user_alice, channel_town_hall)
join_channel(user_bob, channel_town_hall)
users_in_channel(channel_town_hall['PK'])

In [ ]:
channels_for_user(user_alice['PK'])

In [ ]:
find_channel(channel_town_hall['PK']['S'])

## Messages

When we send a message, we create a new item and increment the messages counter:

In [ ]:
def send_message(user, channel, text):
    timestamp = int(time.time())
    message_id = uuid.uuid4()
    
    # create message
    client.put_item(
        TableName="chat",
        Item={
            'PK': channel['PK'],
            'SK': { 'S': f"MSG#{timestamp}#{message_id}" },
            'Msg': { 'S': text },
            'CreatedAt': { 'N': str(timestamp) },
            'UserName': user['Name'],
            'UserId': user['PK']
        }
        )
    
    # increment users
    client.update_item(
        TableName="chat",
        Key = {
            'PK': channel['PK'],
            'SK': channel['PK'],
        },
        ExpressionAttributeValues = {
            ':one': { 'N': '1' }
        },
        UpdateExpression = 'ADD MessageCount :one', 
        ReturnValues = 'UPDATED_NEW'
      )

# AP3
def messages_in_channel(key, limit = 50):
    item = client.query(
        TableName="chat",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :msg)',
        ExpressionAttributeValues={
            ':pk': key,
            ':msg': { 'S': 'MSG#' }
        },
        Limit = limit
    )
    return item['Items'] if 'Items' in item else []
    

send_message(user_bob, channel_town_hall, "Hey there!")
send_message(user_alice, channel_town_hall, "Hello Bob!")
send_message(user_bob, channel_town_hall, "How are you?")
messages_in_channel(channel_town_hall['PK'])

In [ ]:
find_channel(channel_town_hall['PK']['S'])